In [ ]:
# pip install pulp            # PuLP
# $ sudo apt-get install glpk-utils  # GLPK
# $ sudo apt-get install coinor-cbc  # CoinOR

In [2]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
from geopy.distance import great_circle
import random
import pickle

In [3]:
PROJDIR = 'D:/W210_Capstone/co2-storage-fall2021/'
os.chdir(PROJDIR)

In [4]:
# wells = pd.read_csv("data/wells/AllWells_20210915.csv")

# # Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
# wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
#                       (wells['WellType'] == 'DG')  |
#                       (wells['WellType'] == 'GAS') |
#                       (wells['WellType'] == 'LG') ]
# wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
# wells_ids = wells_oilgas['API'].tolist()

In [5]:
pooled_wells = pd.read_csv("data/wells/pool_volumes.csv", dtype = { 'poolID': str })

pools_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pools_ids = pooled_wells['poolID'].tolist()
pools_volumes = pooled_wells['totalco2'].tolist()

In [6]:
# pools_latlon[0:2]
# pools_ids[0:2]
# pools_volumes[0:2]
len(pools_volumes)

94257

In [8]:
emitters = pd.read_csv("data/emitter_volumes.csv")

In [10]:
emitters = emitters[emitters['Cement Production'] > 0]

In [11]:
emitters.shape

(14, 27)

In [15]:
emitters.columns

Index(['Unnamed: 0', 'Facility Id', 'FRS Id', 'Facility Name', 'City', 'State',
       'Primary NAICS Code', 'Reporting Year', 'Industry Type (subparts)',
       'Industry Type (sectors)', 'Unit Name', 'Unit Type',
       'Unit Reporting Method',
       'Unit Maximum Rated Heat Input Capacity (mmBTU/hr)',
       'Unit CO2 emissions (non-biogenic) ', 'Unit Methane (CH4) emissions ',
       'Unit Nitrous Oxide (N2O) emissions ',
       'Unit Biogenic CO2 emissions (metric tons)', 'City2', 'Zip Code',
       'Address', 'County', 'Latitude', 'Longitude', 'Cement Production',
       'Industry Type (subparts)2', 'Industry Type (sectors)2'],
      dtype='object')

In [12]:
# emissions = pd.read_csv(r'data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
# emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
# facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
#                           sheet_name='Direct Emitters', skiprows=3)
# # One Michegan emitter lists its CA headquarters. Retag it as Michigan
# facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

# facilities = facilities[facilities.State == 'CA']

In [13]:
# facilities.shape

In [14]:
# facilitiesB = facilities.rename(inplace=False,
#                                 columns={'Facility Name':'facility_name',
#                                          'City':'city',
#                                          'Primary NAICS Code':'naics_code',
#                                          'Industry Type (subparts)':'industry_subparts',
#                                          'Industry Type (sectors)':'industry_sectors'})

# emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
# #                                               'Facility Name2',
#                                                'city',
#                                                'Zip Code',
#                                                'Address',
#                                                'County',
#                                                'Latitude',
#                                                'Longitude',
# #                                               'Primary NAICS Code2',
#                                                'industry_subparts',
#                                                'industry_sectors']],
#                         how='left', on='Facility Id')
# emsCA2019locagg = emsCA2019loc[['Facility Id','Latitude','Longitude',
#                                 'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
#                                     'Facility Id').agg({'Latitude':'min',
#                                                         'Longitude':'min',
#                                                         'Unit CO2 emissions (non-biogenic)':'sum'})
# emsCA2019locagg.reset_index(inplace=True)
# emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
#                                                   'Latitude':'lat',
#                                                   'Longitude':'lon',
#                                                   'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
# emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
# emsCA2019locagg.head()


In [16]:
facility_latlon = emitters[['Latitude', 'Longitude']].to_records(index=False)
facility_ids = emitters['Facility Id'].tolist()
facility_supply = emitters['Cement Production'].tolist()

In [17]:
# facility_latlon[0:2]
# facility_ids[0:2]
facility_supply[0:2]
# len(facility_supply)

[1123385.34, 1250724.64]

In [20]:
x = time()
pools_distance_matrix = []
for i in range(len(facility_latlon)):
    distances = []
    for j in range(len(pools_latlon)):
        dist = great_circle(facility_latlon[i],
                            pools_latlon[j]).miles * pools_volumes[j]
        distances.append(dist)
    pools_distance_matrix.append(distances)

print(round((time() - x) / 60, 2), 'minutes')

0.66 minutes


In [49]:
# import pickle

# with open('dist_matrix.pkl', 'wb') as handle:
#     pickle.dump(pools_distance_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # with open('filename.pickle', 'rb') as handle:
# #     b = pickle.load(handle)

# # print a == b

In [ ]:
# ## USE THIS CODE TO IMPORT DATA

# file_name = 'data/results/dist_matrix.pkl'
# open_file = open(file_name, "rb")
# loaded_list = pickle.load(open_file)
# open_file.close()



In [22]:
# pools_distance_matrix[0]
print(len(pools_distance_matrix[0]))
print(len(pools_distance_matrix))

94257
14


In [24]:
rt = time()
pools_prob = LpProblem('Unbalanced_Transportation_Problem', LpMinimize)
n_receivers = len(pools_ids)
n_suppliers = len(facility_ids)
supply = facility_supply
demand = pools_volumes
costs = pools_distance_matrix

routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

x = LpVariable.dicts('X', routes, lowBound=0)

pools_prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

for i in range(n_suppliers):
    pools_prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply[i]
    
for j in range(n_receivers):
    pools_prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand[j]
    
# Solving problem
pools_prob.solve()
print(round((time() - rt) / 60, 2), 'minutes')

10.79 minutes


In [26]:
# Also takes forever to run

print('Status', LpStatus[pools_prob.status])

print('Z = {}'.format(value(pools_prob.objective)))

Status Optimal
Z = 1921848803753.8218


In [27]:

for v in pools_prob.variables():
    if (v.varValue > 0):
        print('{} = {}'.format(v.name, v.varValue))

X_(0,_11314) = 1613.9108
X_(0,_11317) = 1727.4278
X_(0,_11385) = 1534.252
X_(0,_11387) = 1462.9921
X_(0,_11393) = 1448.0971
X_(0,_11462) = 1494.6194
X_(0,_11470) = 1490.0068
X_(0,_11474) = 1642.126
X_(0,_11482) = 1565.5512
X_(0,_11490) = 1484.6457
X_(0,_11493) = 1629.9213
X_(0,_11510) = 1533.1365
X_(0,_11529) = 1507.7428
X_(0,_11540) = 1584.9081
X_(0,_11569) = 1485.3018
X_(0,_11580) = 1487.8609
X_(0,_11594) = 1770.7349
X_(0,_11601) = 1553.8714
X_(0,_11615) = 1632.6115
X_(0,_11638) = 1545.0787
X_(0,_11673) = 1595.5153
X_(0,_12120) = 1524.2126
X_(0,_12121) = 1492.9134
X_(0,_12130) = 1722.5066
X_(0,_12139) = 1448.4252
X_(0,_12145) = 1827.4934
X_(0,_12174) = 1566.8201
X_(0,_12226) = 1846.7848
X_(0,_12383) = 1871.8413
X_(0,_12402) = 1879.516
X_(0,_12430) = 1461.2205
X_(0,_12481) = 1574.0814
X_(0,_12487) = 1554.0026
X_(0,_12489) = 1587.4672
X_(0,_12531) = 1493.3727
X_(0,_12562) = 1440.8328
X_(0,_12563) = 1618.838
X_(0,_12613) = 1441.601
X_(0,_12633) = 1675.1969
X_(0,_12636) = 1675.7874
X_(0,

X_(1,_87170) = 2549.3379
X_(1,_87172) = 2774.9328
X_(1,_87173) = 1927.9669
X_(1,_87174) = 2428.0641
X_(1,_87175) = 2152.7634
X_(1,_87177) = 2370.6137
X_(1,_87210) = 2216.6597
X_(1,_87214) = 2073.4262
X_(1,_87215) = 2034.0814
X_(1,_87222) = 2231.3586
X_(1,_87226) = 1842.4885
X_(1,_87229) = 1955.2059
X_(1,_87244) = 2628.18
X_(1,_87252) = 2271.6863
X_(1,_87253) = 3112.0489
X_(1,_87263) = 2999.4718
X_(1,_87289) = 2791.6536
X_(1,_87290) = 2688.0773
X_(1,_87293) = 3172.0616
X_(1,_87299) = 2241.191
X_(1,_87321) = 2476.7098
X_(1,_87324) = 3052.3143
X_(1,_87329) = 3085.8049
X_(1,_87332) = 2152.6565
X_(1,_87375) = 2566.2073
X_(1,_87379) = 2973.8932
X_(1,_87402) = 2447.1129
X_(1,_87422) = 2301.5748
X_(1,_87423) = 2544.685
X_(1,_87440) = 2509.7769
X_(1,_87448) = 2767.5213
X_(1,_87471) = 2641.273
X_(1,_87489) = 3132.9396
X_(1,_87490) = 2576.7717
X_(1,_87496) = 2609.9738
X_(1,_87504) = 2574.2782
X_(1,_87508) = 2883.2677
X_(1,_87527) = 2678.8358
X_(1,_87529) = 2780.9661
X_(1,_87530) = 2511.484
X_(1,_

X_(10,_26314) = 33.005249
X_(10,_26315) = 53.47769
X_(10,_26321) = 41.732283
X_(10,_26326) = 8.3333333
X_(10,_26327) = 2.2309711
X_(10,_2633) = 62.294084
X_(10,_26346) = 37.860892
X_(10,_26355) = 43.700787
X_(10,_2638) = 79.461942
X_(10,_26467) = 11.220472
X_(10,_26473) = 43.110236
X_(10,_2650) = 104.92126
X_(10,_26541) = 43.110236
X_(10,_26549) = 6.6929134
X_(10,_26551) = 5.6248698
X_(10,_26558) = 3.3464567
X_(10,_26595) = 28.28084
X_(10,_2663) = 2.2867301
X_(10,_2664) = 4.1665135
X_(10,_2665) = 50.984252
X_(10,_2666) = 50.984252
X_(10,_2667) = 63.520435
X_(10,_2668) = 112.33596
X_(10,_26701) = 41.010499
X_(10,_26750) = 71.391076
X_(10,_26752) = 37.992126
X_(10,_2685) = 54.063646
X_(10,_2695) = 1.9897605
X_(10,_26967) = 38.32021
X_(10,_26994) = 0.1312336
X_(10,_27) = 1371.1212
X_(10,_27007) = 8.7926509
X_(10,_27009) = 32.611549
X_(10,_27039) = 55.905512
X_(10,_2705) = 101.1811
X_(10,_27170) = 5.839895
X_(10,_2724) = 111.61417
X_(10,_2725) = 111.22047
X_(10,_27289) = 0.85301837
X_(10,_

X_(10,_71815) = 24.907585
X_(10,_71818) = 13.648294
X_(10,_71819) = 7.0209974
X_(10,_71828) = 13.206845
X_(10,_71893) = 21.981627
X_(10,_71908) = 1.6404199
X_(10,_71923) = 25.393701
X_(10,_7193) = 4.5275591
X_(10,_71965) = 28.873256
X_(10,_71996) = 0.065616798
X_(10,_72) = 326.98629
X_(10,_720) = 39.370079
X_(10,_7202) = 63.12336
X_(10,_721) = 40.682415
X_(10,_72130) = 20.90897
X_(10,_72131) = 55.668635
X_(10,_72204) = 3.2152231
X_(10,_72205) = 4.5275591
X_(10,_72280) = 30.586988
X_(10,_72298) = 59.614257
X_(10,_723) = 12.335958
X_(10,_7231) = 33.16238
X_(10,_72377) = 12.270341
X_(10,_72392) = 65.93041
X_(10,_72456) = 8.2020997
X_(10,_7248) = 2.0997375
X_(10,_72518) = 48.425197
X_(10,_72560) = 17.56528
X_(10,_7266) = 12.795276
X_(10,_72677) = 27.23772
X_(10,_7268) = 2.8871391
X_(10,_72753) = 11.286089
X_(10,_72755) = 0.91863517
X_(10,_72757) = 36.614173
X_(10,_72770) = 36.154856
X_(10,_72777) = 31.299213
X_(10,_72778) = 20.472441
X_(10,_72833) = 0.85301837
X_(10,_72834) = 32.874016
X_(

X_(11,_17006) = 138.84514
X_(11,_17017) = 150.32808
X_(11,_17025) = 113.38583
X_(11,_17027) = 80.446194
X_(11,_17048) = 85.761155
X_(11,_17104) = 150.43231
X_(11,_17175) = 125.26247
X_(11,_17218) = 171.98163
X_(11,_17220) = 98.097113
X_(11,_17225) = 110.49869
X_(11,_17235) = 173.8189
X_(11,_17253) = 147.50656
X_(11,_17271) = 139.63255
X_(11,_17272) = 139.37008
X_(11,_17279) = 108.33333
X_(11,_17283) = 169.29134
X_(11,_17311) = 119.75066
X_(11,_17333) = 157.93963
X_(11,_17339) = 150.65617
X_(11,_17345) = 111.48294
X_(11,_17347) = 176.44357
X_(11,_17349) = 105.70866
X_(11,_17352) = 154.19948
X_(11,_17353) = 140.68241
X_(11,_17355) = 90.682415
X_(11,_17367) = 88.910761
X_(11,_17368) = 93.832021
X_(11,_17374) = 88.385827
X_(11,_17376) = 88.713911
X_(11,_17386) = 167.1916
X_(11,_17392) = 177.75591
X_(11,_17430) = 115.81365
X_(11,_17434) = 121.98163
X_(11,_17450) = 73.884514
X_(11,_17457) = 112.34525
X_(11,_17461) = 152.50273
X_(11,_17462) = 83.385704
X_(11,_17472) = 110.43307
X_(11,_17475) 

...

X_(6,_72440) = 918.26027
X_(6,_72452) = 1235.7864
X_(6,_72453) = 1124.1957
X_(6,_72488) = 921.06299
X_(6,_72493) = 1101.2895
X_(6,_72497) = 1130.461
X_(6,_72501) = 1049.8081
X_(6,_72514) = 782.02377
X_(6,_72520) = 1175.0496
X_(6,_72525) = 1300.0884
X_(6,_72526) = 967.23566
X_(6,_72535) = 1232.7179
X_(6,_72540) = 1002.9501
X_(6,_72556) = 843.04904
X_(6,_72561) = 947.42651
X_(6,_72570) = 1022.6738
X_(6,_72573) = 1269.3206
X_(6,_72621) = 1008.6397
X_(6,_72628) = 1129.1018
X_(6,_72643) = 1242.372
X_(6,_72646) = 1105.0887
X_(6,_72674) = 1174.3935
X_(6,_7270) = 756.88976
X_(6,_72704) = 809.16941
X_(6,_72732) = 948.40829
X_(6,_72736) = 1027.4858
X_(6,_72741) = 815.09186
X_(6,_72744) = 1078.6089
X_(6,_7276) = 516.17314
X_(6,_72762) = 881.75853
X_(6,_72768) = 982.08661
X_(6,_72775) = 777.09974
X_(6,_72781) = 963.12336
X_(6,_72791) = 295.91976
X_(6,_7281) = 420.26014
X_(6,_72818) = 901.64042
X_(6,_72829) = 909.71129
X_(6,_72836) = 951.13761
X_(6,_72843) = 1238.6483
X_(6,_72855) = 1006.694
X_(6,_

X_(6,_8825) = 299.8805
X_(6,_8826) = 362.74476
X_(6,_8847) = 239.33317
X_(6,_8849) = 422.04942
X_(6,_8870) = 1117.3248
X_(6,_8879) = 183.00525
X_(6,_8883) = 664.47953
X_(6,_8918) = 713.44932
X_(6,_8954) = 314.70934
X_(6,_8977) = 1258.9697
X_(6,_8991) = 435.18517
X_(6,_9007) = 1406.1746
X_(6,_9048) = 590.0257
X_(6,_9050) = 492.91339
X_(6,_9051) = 370.53806
X_(6,_90587) = 656.15734
X_(6,_9067) = 205.24934
X_(6,_90701) = 608.92213
X_(6,_9071) = 503.54331
X_(6,_9076) = 241.99475
X_(6,_9121) = 596.91601
X_(6,_9142) = 183.88877
X_(6,_9174) = 1230.8399
X_(6,_9218) = 990.32035
X_(6,_92277) = 1254.8095
X_(6,_92586) = 1242.2941
X_(6,_92598) = 488.17087
X_(6,_92743) = 1168.5039
X_(6,_9299) = 308.71637
X_(6,_9324) = 790.94488
X_(6,_9415) = 630.90551
X_(6,_9447) = 1256.5091
X_(6,_9460) = 539.30446
X_(6,_9491) = 262.64727
X_(6,_9644) = 298.88451
X_(6,_9661) = 390.02625
X_(6,_9685) = 1268.8976
X_(6,_9688) = 328.54331
X_(6,_9712) = 553.74016
X_(6,_9733) = 305.11811
X_(6,_9747) = 422.57208
X_(6,_9859) 

X_(7,_81585) = 1036.5627
X_(7,_81588) = 709.7769
X_(7,_81589) = 788.84514
X_(7,_81590) = 888.27801
X_(7,_81596) = 585.9145
X_(7,_81597) = 583.62553
X_(7,_81598) = 697.71878
X_(7,_81599) = 480.24034
X_(7,_81601) = 281.51221
X_(7,_81602) = 356.72993
X_(7,_81603) = 265.36795
X_(7,_81605) = 291.85946
X_(7,_81607) = 333.02953
X_(7,_81609) = 214.89938
X_(7,_81611) = 418.14011
X_(7,_81613) = 250.4504
X_(7,_81614) = 316.36016
X_(7,_81616) = 259.93813
X_(7,_81618) = 784.29699
X_(7,_81619) = 214.63733
X_(7,_81620) = 688.03442
X_(7,_81621) = 1125.0219
X_(7,_81628) = 947.6254
X_(7,_81629) = 920.69387
X_(7,_81631) = 286.81102
X_(7,_81632) = 670.66929
X_(7,_81633) = 684.51444
X_(7,_81634) = 594.65082
X_(7,_81635) = 665.5385
X_(7,_81637) = 983.50791
X_(7,_81638) = 818.89764
X_(7,_81639) = 229.96923
X_(7,_81640) = 445.41792
X_(7,_81641) = 531.32355
X_(7,_81643) = 665.30348
X_(7,_81644) = 389.8294
X_(7,_81647) = 1046.3762
X_(7,_81648) = 1116.0759
X_(7,_81650) = 713.93323
X_(7,_81651) = 1040.5877
X_(7,_

X_(8,_2229) = 898.4252
X_(8,_2246) = 374.6063
X_(8,_2250) = 404.72441
X_(8,_2251) = 653.21522
X_(8,_2252) = 582.34908
X_(8,_2254) = 833.53018
X_(8,_2255) = 306.2336
X_(8,_2256) = 628.41207
X_(8,_2257) = 782.67717
X_(8,_2259) = 343.43832
X_(8,_2260) = 461.15486
X_(8,_2261) = 266.46982
X_(8,_2264) = 365.35433
X_(8,_2265) = 196.85039
X_(8,_2266) = 274.80315
X_(8,_2268) = 971.52231
X_(8,_2269) = 837.07349
X_(8,_2272) = 1245.8005
X_(8,_2274) = 1346.8504
X_(8,_2277) = 1271.9829
X_(8,_2289) = 1025.7218
X_(8,_2292) = 1019.4226
X_(8,_2297) = 1095.5381
X_(8,_2307) = 682.08661
X_(8,_2308) = 976.5748
X_(8,_2310) = 1058.7927
X_(8,_2318) = 677.88714
X_(8,_2328) = 269.22572
X_(8,_2336) = 405.11811
X_(8,_2342) = 655.97113
X_(8,_2348) = 1322.3097
X_(8,_2351) = 1074.2782
X_(8,_2357) = 1091.9291
X_(8,_2365) = 1057.7428
X_(8,_2367) = 724.47507
X_(8,_2372) = 907.60109
X_(8,_2375) = 566.4042
X_(8,_2382) = 575.85302
X_(8,_2390) = 1037.5984
X_(8,_2393) = 792.91339
X_(8,_2394) = 952.69029
X_(8,_2395) = 919.028

X_(9,_5863) = 1447.072
X_(9,_5866) = 457.19853
X_(9,_5867) = 418.4956
X_(9,_5868) = 1086.9588
X_(9,_5873) = 908.13155
X_(9,_5874) = 609.81782
X_(9,_5875) = 224.74863
X_(9,_5878) = 382.49572
X_(9,_5879) = 223.16273
X_(9,_5881) = 134.46515
X_(9,_5882) = 772.3818
X_(9,_5886) = 1297.7034
X_(9,_5887) = 704.80112
X_(9,_5888) = 539.89692
X_(9,_5892) = 1312.8702
X_(9,_5895) = 186.24945
X_(9,_5896) = 266.58886
X_(9,_5897) = 654.11822
X_(9,_5910) = 1240.2619
X_(9,_5911) = 1093.9023
X_(9,_5912) = 1434.5394
X_(9,_5913) = 750.6876
X_(9,_5914) = 729.32641
X_(9,_5915) = 711.11821
X_(9,_5918) = 448.02226
X_(9,_5919) = 356.87314
X_(9,_5920) = 175.25877
X_(9,_5921) = 869.11971
X_(9,_5923) = 1025.2667
X_(9,_5925) = 1104.5408
X_(9,_5926) = 906.98373
X_(9,_5928) = 1077.0797
X_(9,_5929) = 910.43959
X_(9,_5930) = 715.98156
X_(9,_5933) = 579.88267
X_(9,_5934) = 823.10015
X_(9,_5936) = 373.95452
X_(9,_5937) = 1016.6606
X_(9,_5938) = 1506.4349
X_(9,_5939) = 1202.4379
X_(9,_5944) = 740.55242
X_(9,_5954) = 1053.1

In [28]:
volumes_moved = [i.varValue for i in pools_prob.variables()]


In [29]:
total_volume = 0
for i in range(0, len(volumes_moved)):
    total_volume = total_volume + volumes_moved[i]
print(total_volume)

18227678.59586229


In [30]:
volume_matrix = []
k=0
while k < n_suppliers:
    vols = []
    for i in range(n_receivers):
        j = i + k*n_receivers
        vols.append(volumes_moved[j])
    volume_matrix.append(vols)
    k+=1

In [31]:
print(len(volume_matrix))
print(len(volume_matrix[0]))

14
94257


In [33]:
print(len(pools_distance_matrix))
print(len(pools_distance_matrix[0]))

14
94257


In [36]:
results = []
for i in range(n_suppliers):
    f_loc = facility_latlon[i]
    for j in range(n_receivers):
        vol = volume_matrix[i][j]
        if vol > 0:
            w_loc = pools_latlon[j]
            dist = pools_distance_matrix[i][j]
            r = [f_loc, w_loc, dist, vol]
            results.append(r)

In [37]:
file_name = 'data/results/results_cement.pkl'
open_file = open(file_name, "wb")
pickle.dump(results, open_file)
open_file.close()

In [38]:
## USE THIS CODE TO IMPORT DATA

file_name = 'data/results/results_cement.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

